In [1]:
from qiskit_nature.drivers.second_quantization import PySCFDriver
from qiskit_nature.problems.second_quantization.electronic import ElectronicStructureProblem

from qiskit_nature.converters.second_quantization import QubitConverter
from qiskit_nature.mappers.second_quantization import JordanWignerMapper
from qiskit_nature.circuit.library import HartreeFock, UCC
from qiskit.algorithms.optimizers import COBYLA, CG, SLSQP, L_BFGS_B
from qiskit.algorithms import VQE
from qiskit_nature.algorithms.ground_state_solvers.minimum_eigensolver_factories import NumPyMinimumEigensolverFactory
from qiskit import Aer
import numpy as np 
from qiskit_nature.algorithms.ground_state_solvers import GroundStateEigensolver
from qiskit.circuit import Parameter, QuantumCircuit, QuantumRegister

import logging
logging.basicConfig(filename="H2_1.0.log", level=logging.INFO)
logger=logging.getLogger()
logger.setLevel(logging.DEBUG)

coord = 'H 0.0 0.0 0.0; H 0.0 0.0 0.735'
driver = PySCFDriver(atom=coord, charge=0, spin=0, basis='sto3g')
es_problem = ElectronicStructureProblem(driver)

# obtaining qubit Hamiltonian
mapper = JordanWignerMapper()
converter = QubitConverter(mapper=mapper, two_qubit_reduction=False)
second_q_op = es_problem.second_q_ops()
print(second_q_op['ElectronicEnergy'])
qubit_op = converter.convert(second_q_op['ElectronicEnergy'])
print(qubit_op)

es_particle_number = es_problem.grouped_property_transformed.get_property('ParticleNumber')
num_particles = (es_particle_number.num_alpha, es_particle_number.num_beta)
num_spin_orbitals = es_particle_number.num_spin_orbitals
es_energy = es_problem.grouped_property_transformed.get_property('ElectronicEnergy')
#print(es_energy.electronic_energy)
nuclear_repulsion_energy = es_energy.nuclear_repulsion_energy
shift = nuclear_repulsion_energy
print('Number Of Particles: ',num_particles)
print('Number of Spin Orbitals: ', num_spin_orbitals)
print('Nuclear Repulsion Energy: ', nuclear_repulsion_energy)


# initialization of state
init_state = HartreeFock(num_spin_orbitals, num_particles, converter)

#optimizer = L_BFGS_B(maxfun=200000,maxiter=10000)
backend = Aer.get_backend('statevector_simulator')

#Create dummy parametrized circuit for HF calculation
theta = Parameter('a')
n = qubit_op.num_qubits
qc = QuantumCircuit(qubit_op.num_qubits)
qc.rz(theta*0,0)
ansatz = qc
ansatz.compose(init_state, front=True, inplace=True)

#Pass it through VQE
algorithm = VQE(ansatz,quantum_instance=backend)
result = algorithm.compute_minimum_eigenvalue(qubit_op).eigenvalue
print('HF Energy is',np.real(result)+shift)


/tmp/ipykernel_89/2952703543.py:9: NatureDeprecationWarning: The qiskit_nature.algorithms.excited_state_solvers package is deprecated as of version 0.5.0 and will be removed no sooner than 3 months after the release. Instead use the qiskit_nature.second_q.algorithms.excited_state_solvers package.
  from qiskit_nature.algorithms.ground_state_solvers.minimum_eigensolver_factories import NumPyMinimumEigensolverFactory
/tmp/ipykernel_89/2952703543.py:9: NatureDeprecationWarning: The qiskit_nature.algorithms.pes_sampler package is deprecated as of version 0.5.0 and will be removed no sooner than 3 months after the release . Please refer to 'https://github.com/Qiskit/qiskit-nature/issues/750' for additional information regarding 'extrapolators'.
  from qiskit_nature.algorithms.ground_state_solvers.minimum_eigensolver_factories import NumPyMinimumEigensolverFactory


overwrite output file: /tmp/tmpii5kg2rv.log
Fermionic Operator
register length=4, number terms=36
  -1.25633907300325 * ( +_0 -_0 )
+ -0.47189600728114184 * ( +_1 -_1 )
+ -1.25633907300325 * ( +_2 -_2 )
+ -0.47189600728114184 * ( +_3 -_3 )
+ -0.33785507740175813 * ( +_0 +_0 -_0 -_0 )
+ -0.0904 ...
-0.8105479805373275 * IIII
+ 0.17218393261915557 * IIIZ
- 0.2257534922240239 * IIZI
+ 0.17218393261915554 * IZII
- 0.22575349222402397 * ZIII
+ 0.12091263261776627 * IIZZ
+ 0.16892753870087907 * IZIZ
+ 0.04523279994605784 * YYYY
+ 0.04523279994605784 * XXYY
+ 0.04523279994605784 * YYXX
+ 0.04523279994605784 * XXXX
+ 0.16614543256382414 * ZIIZ
+ 0.16614543256382414 * IZZI
+ 0.17464343068300445 * ZIZI
+ 0.12091263261776627 * ZZII
Number Of Particles:  (1, 1)
Number of Spin Orbitals:  4
Nuclear Repulsion Energy:  0.7199689944489797


/tmp/ipykernel_89/2952703543.py:59: DeprecationWarning: The class ``qiskit.algorithms.minimum_eigen_solvers.vqe.VQE`` is deprecated as of qiskit-terra 0.24.0. It will be removed no earlier than 3 months after the release date. Instead, use the class ``qiskit.algorithms.minimum_eigensolvers.VQE``. See https://qisk.it/algo_migration for a migration guide.
  algorithm = VQE(ansatz,quantum_instance=backend)


HF Energy is -1.116998996754004


In [5]:
second_q_op = es_problem.second_q_ops()
print(second_q_op['ElectronicEnergy'])

overwrite output file: /tmp/tmputvgkt0r.log
Fermionic Operator
register length=4, number terms=36
  -1.25633907300325 * ( +_0 -_0 )
+ -0.47189600728114184 * ( +_1 -_1 )
+ -1.25633907300325 * ( +_2 -_2 )
+ -0.47189600728114184 * ( +_3 -_3 )
+ -0.33785507740175813 * ( +_0 +_0 -_0 -_0 )
+ -0.0904 ...


In [2]:
print(driver)

In [3]:
es_problem

AttributeError: 'ElectronicStructureProblem' object has no attribute 'hamiltonian'

In [5]:
from qiskit_nature.second_q.hamiltonians import ElectronicEnergy

hamiltonian = ElectronicEnergy(integrals, constants={"nuclear_repulsion_energy": 1.0})

NameError: name 'integrals' is not defined